In [5]:
import random as rn
import numpy as np
from cryptography.hazmat.primitives import hashes

# BIKE
Vamos agora falar um pouco do mecanismo BIKE, o unico dos 3 propostos para o trabalho que não passou à ultima faze do concurso por questões de eficiencia.
Quando consultamos a documentação reparámos desde logo que das 3 é talvez a menos detalhada (na nossa opinião) pelo que tivemos bastantes dificuldade na sua implementação. Vimos que o professor tem na dropbox da disciplina o código deste mecanismo com o decoder Bit flip. Decidimos então implementar o código disponibilizado, com algumas alterações para nossa compreensão e para que esteja num contexto de classes e depois tentámos trocar o algoritmos Bit Flip pelo Black-Gray Decoder visto que este é o decoder proposto no mais recente documento de especificação.

In [6]:
class BIKE():
    def __init__(self):
        self.r = 257
        self.n = 2 * self.r
        self.t = 16

        self.F = GF(2)
        self.u = self.F(1)
        self.z = self.F(0)

        R = PolynomialRing(self.F, 'a') ; a = R.gen()
        self.Rr = R.quotient(a ^ self.r - 1 , 'x') ; self.x = self.Rr.gen()

        self.Vn  = VectorSpace(self.F, self.n)
        self.Vr  = VectorSpace(self.F, self.r)
        self.Vq  = VectorSpace(QQ, self.r)
        self.Mr  = MatrixSpace(self.F, self.n, self.r)


    # sparse polynomials of size r
    # produz sempre um polinómio mónico com o último coeficiente igual a 1
    # o parametro "sparse > 0"  é o numero de coeficientes não nulos sem contar com o primeiro e o ultimo
    def sparse_pol(self, sparse=3):
        coeffs = [1] * sparse + [0] * (self.r - 2 - sparse)
        rn.shuffle(coeffs)
        return self.Rr([1] + coeffs + [1])

    # Noise
    # produz um par de polinomios dispersos de tamanho "r" com um dado número total de erros "t"
    def noise(self):
        el = [self.u] * self.t + [self.z] * (self.n - self.t)
        rn.shuffle(el)  
        return (self.Rr(el[:self.r]), self.Rr(el[self.r:]))
    

    def expand(self, f):
        fl = f.list(); ex = self.r - len(fl)
        return self.Vr(fl + [self.z] * ex)

    def expand2(self, code):
        (f0,f1) = code
        f = self.expand(f0).list() + self.expand(f1).list()
        return self.Vn(f)

    def unexpand2(self, vec):
        a = vec.list()
        return (self.Rr(a[:self.r]), self.Rr(a[self.r:]))

    
    def rot(self, h):
        v = self.Vr() ; v[0] = h[-1]
        for i in range(self.r - 1):
            v[i + 1] = h[i]
        return v

    def Rot(self, h):
        M = Matrix(self.F, self.r, self.r) ; M[0] = self.expand(h)
        for i in range(1, self.r):
            M[i] = self.rot(M[i - 1])
        return M

    
    def mk_key(self, a):
        uu  = np.packbits(list(map(lift,self.expand2(a))))
        hsh = hashes.Hash(hashes.SHAKE256(int(256)))
        hsh.update(uu)
        return hsh.finalize()


    def mask(self, b , v):                                 ## 
        return b.pairwise_product(v)

    def hamm(self, b):                                     ## peso de Hamming
        return sum([1 if a == self.u else 0 for a in b])

    # Uma implementação do algoritmo Bit Flip sem quaisquer optimizações
    def BF(self, H, code, synd, errs=0):
        mycode   = code   # F2 cipher
        mysynd   = synd   # Maximal syndrome weight
        cnt_iter = self.r # Maximal number of iterations

        while cnt_iter > 0 and self.hamm(mysynd) > errs:
            cnt_iter   = cnt_iter - 1
            
            unsats     = [self.hamm(self.mask(mysynd, H[i])) for i in range(self.n)]
            max_unsats = max(unsats)
            
            for i in range(self.n):
                if unsats[i]  == max_unsats:
                    mycode[i] += self.u               ## bit-flip
                    mysynd    += H[i]
                    
        return mycode


    def BitFlipIter(self, synd, code, max_unsat, H, errs=0):
        mycode   = code             # F2 cipher
        mysynd   = synd             # Maximal syndrome weight

        black  = [0] * self.n 
        gray   = [0] * self.n
        unsats = [self.hamm(self.mask(mysynd, H[i])) for i in range(self.n)]

        for i in range(self.n):
            if unsats[i] == max_unsat:
                mycode[i] += self.u
                black[i]   = self.u
                mysynd    += H[i]
            elif unsats[i] >= max_unsat - 3:
                gray[i] = self.u
                mysynd += H[i]
            
        return (mysynd, mycode, black, gray)
    
    def BitFlipMaskedIter(self, synd, code, mask, max_unsat, H):
        mycode = code
        mysynd = synd
        unsats = [self.hamm(self.mask(mysynd, H[i])) for i in range(self.n)]
        for i in range(self.n):
            if unsats[i] == max_unsat: # ==?
                mycode[i] += mask[i]
                mysynd    += H[i]

        return (synd, code)

    def BGF(self, H, code, synd, errs=0):
        mycode   = code   # F2 cipher
        mysynd   = synd   # Maximal syndrome weight
        cnt_iter = self.r # Maximal number of iterations

        while cnt_iter > 0 and self.hamm(mysynd) > errs:
            cnt_iter   = cnt_iter - 1
            
            unsats   = [self.hamm(self.mask(mysynd, H[i])) for i in range(self.n)]
            max_unsat = max(unsats)
            
            (mysynd, mycode, black, gray) = self.BitFlipIter(mysynd, mycode, max_unsat, H)
            (mysynd, mycode) = self.BitFlipMaskedIter(mysynd, mycode, black, max_unsat, H)
            (mysynd, mycode) = self.BitFlipMaskedIter(mysynd, mycode, gray, max_unsat, H)

        return mycode
            

In [7]:
class BIKE_PKE(BIKE):
    def key_generation(self):
        while True:
            h0 = self.sparse_pol(); h1 = self.sparse_pol()
            if h0 != h1 and h0.is_unit() and h1.is_unit():
                break
        
        secret_key = (h0, h1)
        public_key = (1, h0 / h1) 
        return public_key, secret_key

    def encryption(self, public_key, message, e10=None, e11=None):
        g0, g1 = public_key
        if e10 != None and e11 != None:
            e0 = e10
            e1 = e11
        else:
            e0, e1 = self.noise()


        return (message * g0 + e0, message * g1 + e1)

    def decryption(self, secret_key, cifra):
        code = self.expand2(cifra)

        (h0, h1) = secret_key
        H        = block_matrix(2, 1, [self.Rot(h0), self.Rot(h1)])
        synd     = code * H

        cw = self.BF(H, code, synd)
        (cw0, cw1) = self.unexpand2(cw)

        return (cw0, cw1)

    
    def decryption_BGF(self, secret_key, cifra):
        code = self.expand2(cifra)

        (h0, h1) = secret_key
        H        = block_matrix(2, 1, [self.Rot(h0), self.Rot(h1)])
        synd     = code * H

        cw = self.BGF(H, code, synd)
        (cw0, cw1) = self.unexpand2(cw)

        return (cw0, cw1)

In [8]:
bike = BIKE()
message = bike.Rr.random_element()  

In [9]:
bg = BIKE_PKE()

public_key, secret_key = bg.key_generation()
cifra          = bg.encryption(public_key, message)
message_dec, _ = bg.decryption_BGF(secret_key, cifra)

message == message_dec

False

In [10]:
pke = BIKE_PKE()

public_key, secret_key = pke.key_generation()
cifra          = pke.encryption(public_key, message)
message_dec, _ = pke.decryption(secret_key, cifra)

message == message_dec

True

In [11]:
class BIKE_KEM(BIKE_PKE):
    def encapsulation(self, public_key, message):
        (g0, g1) = public_key
        (e0,e1)  = self.noise()

        key = self.mk_key((e0, e1))
        enc = self.encryption(public_key, message, e0, e1)
        
        return (key, enc)

    def decapsulation(self, secret_key, cifra):
        (cw0, cw1) = self.decryption(secret_key, cifra)
        enc0, enc1 = cifra
        
        return self.mk_key((cw0 + enc0 , cw1 + enc1))

        

In [12]:
kem = BIKE_KEM()
public_key, secret_key = kem.key_generation()
key1, enc = kem.encapsulation(public_key, message)
key2      = kem.decapsulation(secret_key, enc)

key1 == key2

True